# Set up

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from pathlib import Path
import io

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [ ]:
# Download a file based on its file ID.
# https://drive.google.com/file/d/1XsRqaT1TUUOs4zOI6Lr6tPTZ3WGj0Dav/view?usp=sharing
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz

#https://drive.google.com/file/d/1tSAuWjJ93Y_x0YNzsqAxXjs4o45DVm_E/view?usp=sharing
file_id = '1tSAuWjJ93Y_x0YNzsqAxXjs4o45DVm_E' #cdc_death_counts_by_sex_age_state.csv dataset

file_obj = drive.CreateFile({'id': file_id})
content_string = file_obj.GetContentString()

covid = pd.read_csv(io.StringIO(content_string))
covid

,Data As Of,Start Date,End Date,Group,Year,Month,State,Sex,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
0,03/23/2022,01/01/2020,03/19/2022,By Total,NaN,NaN,United States,All Sexes,All Ages,974282.0,7520550.0,855731.0,504882.0,10684.0,1334231.0,NaN
1,03/23/2022,01/01/2020,03/19/2022,By Total,NaN,NaN,United States,All Sexes,Under 1 year,242.0,41777.0,495.0,28.0,28.0,736.0,NaN
2,03/23/2022,01/01/2020,03/19/2022,By Total,NaN,NaN,United States,All Sexes,0-17 years,933.0,74176.0,1460.0,252.0,204.0,2342.0,NaN
3,03/23/2022,01/01/2020,03/19/2022,By Total,NaN,NaN,United States,All Sexes,1-4 years,113.0,7791.0,293.0,29.0,70.0,447.0,NaN
4,03/23/2022,01/01/2020,03/19/2022,By Total,NaN,NaN,United States,All Sexes,5-14 years,296.0,12438.0,424.0,96.0,82.0,705.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85369,03/23/2022,03/01/2022,03/19/2022,By Month,2022.0,3.0,Puerto Rico,Female,50-64 years,0.0,0.0,0.0,0.0,0.0,0.0,NaN
85370,03/23/2022,03/01/2022,03/19/2022,By Month,2022.0,3.0,Puerto Rico,Female,55-64 years,0.0,0.0,0.0,0.0,0.0,0.0,NaN
85371,03/23/2022,03/01/2022,03/19/2022,By Month,2022.0,3.0,Puerto Rico,Female,65-74 years,0.0,0.0,0.0,0.0,0.0,0.0,NaN
85372,03/23/2022,03/01/2022,03/19/2022,By Month,2022.0,3.0,Puerto Rico,Female,75-84 years,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [ ]:
onlyMonth = covid[(covid['Group'] == 'By Month')] #using only month for the data


In [ ]:
onlyMonth['Age Group'].unique()

array(['All Ages', 'Under 1 year', '0-17 years', '1-4 years',
       '5-14 years', '15-24 years', '18-29 years', '25-34 years',
       '30-39 years', '35-44 years', '40-49 years', '45-54 years',
       '50-64 years', '55-64 years', '65-74 years', '75-84 years',
       '85 years and over'], dtype=object)

# Feature Engineering


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Encoding Age
onlyMonth["Age_encoded"] = label_encoder.fit_transform(onlyMonth["Age Group"])
# onlyMonth.head()
label_mapping_age = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
original_values_age = {label_mapping_age[label]: label for label in label_mapping_age}
# original_values_age
onlyMonth = onlyMonth.drop(columns=['Age Group', 'Sex', 'Group', 'Footnote', 'Pneumonia and COVID-19 Deaths', 'Pneumonia, Influenza, or COVID-19 Deaths'])

# onlyMonth



<ipython-input-5-483292a4c92f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onlyMonth["Age_encoded"] = label_encoder.fit_transform(onlyMonth["Age Group"])


In [ ]:
#encoding State

label_encoder = LabelEncoder()
onlyMonth["State_encoded"] = label_encoder.fit_transform(onlyMonth["State"])

label_mapping_state = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
original_values_state = {label_mapping_state[label]: label for label in label_mapping_state}
onlyMonth = onlyMonth.drop(columns=['State'])
original_values_state

{0: 'Alabama',
 1: 'Alaska',
 2: 'Arizona',
 3: 'Arkansas',
 4: 'California',
 5: 'Colorado',
 6: 'Connecticut',
 7: 'Delaware',
 8: 'District of Columbia',
 9: 'Florida',
 10: 'Georgia',
 11: 'Hawaii',
 12: 'Idaho',
 13: 'Illinois',
 14: 'Indiana',
 15: 'Iowa',
 16: 'Kansas',
 17: 'Kentucky',
 18: 'Louisiana',
 19: 'Maine',
 20: 'Maryland',
 21: 'Massachusetts',
 22: 'Michigan',
 23: 'Minnesota',
 24: 'Mississippi',
 25: 'Missouri',
 26: 'Montana',
 27: 'Nebraska',
 28: 'Nevada',
 29: 'New Hampshire',
 30: 'New Jersey',
 31: 'New Mexico',
 32: 'New York',
 33: 'New York City',
 34: 'North Carolina',
 35: 'North Dakota',
 36: 'Ohio',
 37: 'Oklahoma',
 38: 'Oregon',
 39: 'Pennsylvania',
 40: 'Puerto Rico',
 41: 'Rhode Island',
 42: 'South Carolina',
 43: 'South Dakota',
 44: 'Tennessee',
 45: 'Texas',
 46: 'United States',
 47: 'Utah',
 48: 'Vermont',
 49: 'Virginia',
 50: 'Washington',
 51: 'West Virginia',
 52: 'Wisconsin',
 53: 'Wyoming'}

In [ ]:
onlyMonth = onlyMonth.drop(columns=['Data As Of', 'Start Date', 'End Date'])
onlyMonth

,Year,Month,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Influenza Deaths,Age_encoded,State_encoded
11016,2020.0,1.0,5.0,264658.0,17909.0,2125.0,15,46
11017,2020.0,1.0,0.0,1781.0,41.0,8.0,16,46
11018,2020.0,1.0,0.0,2963.0,90.0,63.0,0,46
11019,2020.0,1.0,0.0,315.0,22.0,18.0,1,46
11020,2020.0,1.0,0.0,471.0,21.0,29.0,9,46
...,...,...,...,...,...,...,...,...
85369,2022.0,3.0,0.0,0.0,0.0,0.0,10,40
85370,2022.0,3.0,0.0,0.0,0.0,0.0,11,40
85371,2022.0,3.0,0.0,0.0,0.0,0.0,12,40
85372,2022.0,3.0,0.0,0.0,0.0,0.0,13,40


#Modeling

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [ ]:
X_train = onlyMonth[onlyMonth['Year'] < 2022].dropna().sample(frac=1).reset_index(drop=True)
y_train = X_train['Age_encoded']
X_train = X_train.drop(columns=['Age_encoded'])

In [ ]:
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
X_test = onlyMonth[onlyMonth['Year'] >= 2022].dropna().sample(frac=1).reset_index(drop=True)
y_test = X_test['Age_encoded']
X_test = X_test.drop(columns = ['Age_encoded'])

In [ ]:
y_pred_train = np.round(model.predict(X_train))
y_pred_test = np.round(model.predict(X_test))

In [ ]:
from sklearn.metrics import accuracy_score

training_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy - Linear Regression% : ", training_accuracy*100)

test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy - Linear Regression% (Year 2022 data) : ", test_accuracy*100)

Training Accuracy - Linear Regression% :  5.509387503847337
Test Accuracy - Linear Regression% (Year 2022 data) :  7.1714476317545035


In [ ]:
#ridge Regression
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)

y_pred_train = np.round(ridge.predict(X_train))
y_pred_test = np.round(ridge.predict(X_test))

training_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy - Ridge Regression (alpha=1)% : ", training_accuracy*100)

test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy - Ridge Regression (alpha=1)% (Year 2022 data) : ", test_accuracy*100)

Training Accuracy - Ridge Regression (alpha=1)% :  5.509387503847337
Test Accuracy - Ridge Regression (alpha=1)% (Year 2022 data) :  7.1714476317545035


In [ ]:
ridge = Ridge(alpha=0.5)
ridge.fit(X_train, y_train)

y_pred_train = np.round(ridge.predict(X_train))
y_pred_test = np.round(ridge.predict(X_test))

training_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy - Ridge Regression (alpha=0.5)% : ", training_accuracy*100)

test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy - Ridge Regression (alpha=0.5)% (Year 2022 data) : ", test_accuracy*100)

Training Accuracy - Ridge Regression (alpha=0.5)% :  5.509387503847337
Test Accuracy - Ridge Regression (alpha=0.5)% (Year 2022 data) :  7.1714476317545035


In [ ]:
#random forest

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred_train = np.round(rf.predict(X_train))
y_pred_test = np.round(rf.predict(X_test))

training_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy - Random Forest (n=100)% : ", training_accuracy*100)

test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy - Random Forest (n=100)% (Year 2022 data) : ", test_accuracy*100)

Training Accuracy - Random Forest (n=100)% :  54.13588796552785
Test Accuracy - Random Forest (n=100)% (Year 2022 data) :  13.642428285523684


In [ ]:
rf = RandomForestRegressor(n_estimators=500, random_state=42)
rf.fit(X_train, y_train)

y_pred_train = np.round(rf.predict(X_train))
y_pred_test = np.round(rf.predict(X_test))

training_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy - Random Forest (n=500)% : ", training_accuracy*100)

test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy - Random Forest (n=500)% (Year 2022 data) : ", test_accuracy*100)

Training Accuracy - Random Forest (n=500)% :  53.81271160357033
Test Accuracy - Random Forest (n=500)% (Year 2022 data) :  13.942628418945963


In [ ]:
#logistic Regression

from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

y_pred_train = np.round(logistic_model.predict(X_train))
y_pred_test = np.round(logistic_model.predict(X_test))

training_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy - Logistic Regression% : ", training_accuracy*100)

test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy - Logistic Regression% (Year 2022 data) : ", test_accuracy*100)

Training Accuracy - Logistic Regression% :  16.389658356417357
Test Accuracy - Logistic Regression% (Year 2022 data) :  13.075383589059372


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#SVM

from sklearn.svm import SVC

svm_model = SVC(kernel='sigmoid', C=10)
svm_model.fit(X_train, y_train)

y_pred_train = np.round(svm_model.predict(X_train))
y_pred_test = np.round(svm_model.predict(X_test))

training_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy - SVM% : ", training_accuracy*100)

test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy - SVM% (Year 2022 data) : ", test_accuracy*100)

Training Accuracy - SVM% :  16.08187134502924
Test Accuracy - SVM% (Year 2022 data) :  12.241494329553035
